# import modules

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
import time
import os
import csv

# start a selenium driven browser

In [2]:
# get brand key word
def get_key_word(driver, web_url):
    driver.get(web_url)
    time.sleep(0.2)
    search_terms = input('Please enter a brand.\t')
    print(f'search for the terms: \t{search_terms}')
    # search the key words
    search_field = driver.find_element_by_id('keywords')
    search_field.send_keys(search_terms)
    search_field.send_keys(Keys.RETURN)
    # get search terms results
    brands = driver.find_elements_by_class_name('jss5')
    result_lst = [ i.find_element_by_class_name('MuiTypography-root').text for i in brands]
    if 'for iOS'in str(result_lst):
        print('Please try another search tearms for brand.')
        get_key_word(driver, web_url)
    return result_lst

In [3]:
#narrow down to the specific brand
def get_brand(driver, web_url, result_lst):
    def check_brand(result_lst):
        for i in result_lst:
            yield(i)
    brands = check_brand(result_lst)
    correct = 'F'
    while correct == 'F':
        try:
            brand_candidate = next(brands)
            correct = input(f'If {brand_candidate} the brand you are looking for? T/F:\t').upper()
            if correct =='T':
                target_brand = str(brand_candidate)
                return target_brand 
                break
        except: 
            print('Please try another search tearms for brand.')
            get_key_word(driver, web_url)
            target_brand = get_brand(driver, web_url, result_lst)
            return target_brand 
            break
    
        
     


In [4]:
# search for the dish of the brand found in get_brand()
def get_key_word_checked(driver, web_url, target_brand):
    driver.get(web_url)
    time.sleep(0.2)
    meal_terms = input('Please enter the main meal ingrediant.\t').upper()
    search_terms = target_brand + ' ' + meal_terms
    print(f'search for the terms: {search_terms}')
    # search the key words
    search_field = driver.find_element_by_id('keywords')
    search_field.send_keys(search_terms)
    search_field.send_keys(Keys.RETURN)
    # get search terms results
    foods = driver.find_elements_by_class_name('jss374')
    food_lst = [ i.text for i in foods]
    if 'for iOS'in str(food_lst):
        print('Please try another search tearms for brand.')
        get_key_word_checked(driver, web_url, target_brand)
    return food_lst

In [5]:
# narrow down to the specific dish and get the dish's info link
def get_food(driver, web_url, target_brand, food_lst):
    count=0
    def check_food(food_lst):
        for i in food_lst:
            yield(i)
    foods = check_food(food_lst)
    correct = 'F'
    while correct == 'F':
        try:
            food_candidate = next(foods)
            correct = input(f'If {food_candidate} the meal you are looking for? T/F:\t').upper()
            if correct =='T':
                target_food = str(food_candidate)
                url = driver.find_elements_by_class_name('MuiListItem-button')[count].get_attribute('href')
                target_food_url = (url)
                return target_food_url, target_food
                break
            else:
                count=count+1
        except: 
            print('Please try another search tearms for meal.')
            get_key_word_checked(driver, web_url, target_brand)
            target_food_url, target_food = get_food(driver, web_url, target_brand, food_lst)
            return target_food_url, target_food
            break
           
            
    


In [6]:
def get_food_info(driver, target_food_url):
    driver.get(target_food_url)
    infos = driver.find_element_by_class_name('jss374')
    infos = infos.text.split('\n')
    
    calories = infos[0].split(' ')[0]
    fat = infos[6].split(' ')[0]
    carbs = infos[8].split(' ')[0]
    fiber = infos[10].split(' ')[0]
    protein = infos[12].split(' ')[0]
    
    
    return calories, fat, carbs, fiber, protein

In [7]:
def save_result(target_brand, target_food, calories, fat, carbs, fiber, protein):
    
    meal_translator = {'B':'Breakfast','L':"Lunch",'D':'Dinner','O':"Others"}
    if os.path.isfile('food_recoder.csv') == False:
        with open('food_recoder.csv', 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['Time', 'Meal_Time' , 'Brand', 'Food', 'Calories', 'Fat', 'Carbs', 'Fiber', 'Protein'])
    
    time = input('Please enter the date. mm/dd/yyyy.\t')
    print('Please enter the meal time.')
    meal_time = input('B---Breakfast\nL---Lunch\nD---Dinner\nO---Others').upper()
    meal_time_transfered = meal_translator[meal_time]
    with open('food_recoder.csv', 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow([time, meal_time_transfered, target_brand, target_food, calories, fat, carbs, fiber, protein])



In [8]:
def run_again():
    run = input('Have another meal to record? T/F \t').upper()
    if run == 'T':
        main()
    else:
        print('Thanks for using this software.')
        

In [9]:
def start_program():
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    driver = webdriver.Chrome(options=options)
    return driver

In [10]:
def main(): 
    web_url = 'https://www.calorieking.com/us/en/'
    driver = start_program()
    result_lst = get_key_word(driver, web_url)
    target_brand = get_brand(driver, web_url, result_lst)
    food_lst = get_key_word_checked(driver, web_url, target_brand)
    target_food_url, target_food = get_food(driver, web_url, target_brand, food_lst)
    calories, fat, carbs, fiber, protein = get_food_info(driver, target_food_url)
    save_result(target_brand, target_food, calories, fat, carbs, fiber, protein)
    run_again()

In [11]:
main()

Please enter a brand.	applebee
search for the terms: 	applebee
If Applebee's the brand you are looking for? T/F:	t
Please enter the main meal ingrediant.	steak
search for the terms: Applebee's STEAK
If Sirloin Steak (8 oz), without sides the meal you are looking for? T/F:	t
Please enter the date. mm/dd/yyyy.	01/02/2021
Please enter the meal time.
B---Breakfast
L---Lunch
D---Dinner
O---Othersd
Have another meal to record? T/F 	f
Thanks for using this software.


In [12]:
# check for results
import pandas as pd
record = pd.read_csv('food_recoder.csv')
record.head()

,Time,Meal_Time,Brand,Food,Calories,Fat,Carbs,Fiber,Protein
0,01/02/2021,Breakfast,Popeye's,Classic or Spicy Chicken Sandwich,700,42,50,2,28
1,01/02/2021,Dinner,Applebee's,"Sirloin Steak (8 oz), without sides",280,12,1,1,43
